In [2]:
# Must be included at the beginning of each new notebook. Remember to change the app name.
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Iteration 4').getOrCreate()

# If you're getting an error with numpy, please type 'sudo pip install numpy --user' into the EC2 console.
from pyspark.ml.regression import LinearRegression

# 2. DATA UNDERSTANDING

In [3]:
# Load the Data 
bank = spark.read.csv("bank.csv", inferSchema=True, header=True, nullValue='unknown')

In [4]:
# Check the number of rows and columns
bank.count(), len(bank.columns)

(41188, 21)

In [5]:
# View the top 5 observation in a Pandas data type
bank.limit(5).toPandas()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,takers
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,0.042361,93994.0,-36.4,4857.0,5191.0,no
1,57,services,married,high.school,None,no,no,telephone,may,mon,...,1,999,0,nonexistent,0.042361,93994.0,-36.4,4857.0,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,0.042361,93994.0,-36.4,4857.0,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,0.042361,93994.0,-36.4,4857.0,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,0.042361,93994.0,-36.4,4857.0,5191.0,no


In [6]:
# Check object type
print(type(bank))

# Cek column types
print(bank.printSchema())

<class 'pyspark.sql.dataframe.DataFrame'>
root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- emp_var_rate: double (nullable = true)
 |-- cons_price_idx: double (nullable = true)
 |-- cons_conf_idx: double (nullable = true)
 |-- euribor3m: double (nullable = true)
 |-- nr_employed: double (nullable = true)
 |-- takers: string (nullable = true)

None


In [7]:
# Descriptive Statistics of the numerical columns
bank.describe().toPandas()

,summary,age,job,marital,education,default,housing,loan,contact,month,...,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,takers
0,count,41188,40858,41108,39457,32591,40198,40198,41188,41188,...,41188,41188,41188,41188,41188,41188,41188,41188,41188,41188
1,mean,40.02406040594348,None,None,None,None,None,None,None,None,...,2.567592502670681,962.4754540157328,0.17296299893172767,None,-0.6517393316357063,85393.73800676703,-40.50260027191399,3179.731503495394,1152.9059172064758,None
2,stddev,10.421249980934045,None,None,None,None,None,None,None,None,...,2.770013542902322,186.91090734474153,0.49490107983929055,None,1.0248699440001923,26496.779264029818,4.628197856174574,2087.2625209512007,1946.0827256394978,None
3,min,17,admin.,divorced,basic.4y,no,no,no,cellular,apr,...,1,0,0,failure,-3.4,3.876388889,-50.8,0.004861111,206.7958333,no
4,max,98,unemployed,single,university.degree,yes,yes,yes,telephone,sep,...,56,999,7,success,0.044444444,94767.0,-26.9,5045.0,5191.0,yes


In [9]:
# Check Missing Values
from pyspark.sql.functions import *
import pyspark.sql.functions as f

bank_agg = bank.agg(*[f.count(f.when(f.isnull(c), c)).alias(c) for c in bank.columns])
from functools import reduce
bank_agg_col = reduce(
    lambda a, b: a.union(b),
    (
        bank_agg.select(f.lit(c).alias("Column_Name"), f.col(c).alias("Count_Missing")) 
        for c in bank_agg.columns
    )
)

bank_agg_col = bank_agg_col.withColumn('Percent_Missing (%)', expr('Count_Missing/41188*100'))
bank_agg_col = bank_agg_col.select(['Column_Name', 'Count_Missing', 
                                    round('Percent_Missing (%)', 3).alias("Percent_Missing (%)")])
bank_agg_col.show()

+--------------+-------------+-------------------+
|   Column_Name|Count_Missing|Percent_Missing (%)|
+--------------+-------------+-------------------+
|           age|            0|                0.0|
|           job|          330|              0.801|
|       marital|           80|              0.194|
|     education|         1731|              4.203|
|       default|         8597|             20.873|
|       housing|          990|              2.404|
|          loan|          990|              2.404|
|       contact|            0|                0.0|
|         month|            0|                0.0|
|   day_of_week|            0|                0.0|
|      duration|            0|                0.0|
|      campaign|            0|                0.0|
|         pdays|            0|                0.0|
|      previous|            0|                0.0|
|      poutcome|            0|                0.0|
|  emp_var_rate|            0|                0.0|
|cons_price_idx|            0| 